In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates. All rights reserved.

# Render a textured mesh

This tutorial shows how to:
- load a mesh and textures from an `.obj` file.
- set up a renderer
- render the mesh
- vary the rendering settings such as lighting and camera position
- use the batching features of the pytorch3d API to render the mesh from different viewpoints

## 0. Install and Import modules

Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [2]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("1.13.", "2.0.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 324.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61406 sha256=6047311d25de74e7ef2e79bf3e46c2f150448019faf853fd99709bc17d466885
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=62e93b2b160e8ffda57961ce5bd8a046fc057bb68db79bca49b2f805ca9b5853
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py310_cu118_pyt201/download.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 152.9 MB/s eta 0:00:00


In [3]:
import os
import torch
import matplotlib.pyplot as plt

# Util function for loading meshes
from pytorch3d.io import load_objs_as_meshes, load_obj

# Data structures and functions for rendering
from pytorch3d.structures import Meshes
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.vis.texture_vis import texturesuv_image_matplotlib
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    DirectionalLights,
    Materials,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader,
    TexturesUV,
    TexturesVertex
)

# add path for demo utils functions
import sys
import os
sys.path.append(os.path.abspath(''))

If using **Google Colab**, fetch the utils file for plotting image grids:

In [4]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
from plot_image_grid import image_grid

--2023-08-10 03:18:08--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/main/docs/tutorials/utils/plot_image_grid.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘plot_image_grid.py’

plot_image_grid.py  100%[===================>]   1.57K  --.-KB/s    in 0s      

2023-08-10 03:18:08 (28.1 MB/s) - ‘plot_image_grid.py’ saved [1608/1608]



OR if running **locally** uncomment and run the following cell:

In [5]:
# from utils import image_grid

In [6]:
!ls

plot_image_grid.py  __pycache__  sample_data


In [7]:
from google.colab import drive

drive.mount('/content/drive')

import os

os.chdir("/content/drive/MyDrive/Colab Notebooks")


Mounted at /content/drive


In [8]:
!ls

1_b.gif			      q_5-1_pc_union.gif
2.1.gif			      q_5-2.gif
3_b.gif			      rendering_generic_3d_representations.ipynb
data			      rendering_parametric_and_implicit.ipynb
deform_form.ipynb	      rendering_point_clouds_from_rgbd.ipynb
plotly_rendered_meshes.ipynb  rendering_texture_obj.ipynb
q_5-1_pc1.gif		      render_tetrahedron_and_retexturing.ipynb
q_5-1_pc2.gif		      render_texture_mesh_and_dolly_zoom.ipynb


In [9]:
if torch.cuda.is_available():
  device=torch.device("cuda:0")
else:
  device=torch.device("cpu")
  print("gpu is not available")


In [10]:
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith(("1.13.", "2.0.")) and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

In [11]:
from torch.cuda import is_available

from pytorch3d.renderer.mesh.textures import TexturesAtlas
import torch

from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.loss import(
    chamfer_distance,
    mesh_edge_loss,
    mesh_normal_consistency,
    mesh_laplacian_smoothing
)

from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    SoftPhongShader,
    MeshRenderer,
    MeshRasterizer,
    RasterizationSettings,
    TexturesAtlas,
    TexturesVertex,
    PointsRenderer,
    AlphaCompositor,
    NormWeightedCompositor,
    PointsRasterizationSettings,
    PointsRenderer,
    PulsarPointsRenderer,
    PointsRasterizer,
)

import numpy as np

from tqdm.notebook import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D

In [12]:
!ls

1_b.gif			      q_5-1_pc_union.gif
2.1.gif			      q_5-2.gif
3_b.gif			      rendering_generic_3d_representations.ipynb
data			      rendering_parametric_and_implicit.ipynb
deform_form.ipynb	      rendering_point_clouds_from_rgbd.ipynb
plotly_rendered_meshes.ipynb  rendering_texture_obj.ipynb
q_5-1_pc1.gif		      render_tetrahedron_and_retexturing.ipynb
q_5-1_pc2.gif		      render_texture_mesh_and_dolly_zoom.ipynb


In [13]:
num_samples=200

phi = torch.linspace(0, 2 * np.pi, num_samples)
theta = torch.linspace(0, 2 * np.pi, num_samples)

print(phi.shape)
print(theta.shape)
# Densely sample phi and theta on a grid
Phi, Theta = torch.meshgrid(phi, theta)
print(Phi.shape)
print(Theta.shape)


torch.Size([200])
torch.Size([200])
torch.Size([200, 200])
torch.Size([200, 200])


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [14]:
def get_points_renderer(
    image_size=512, device=None, radius=0.01, background_color=(1, 1, 1)
):
    """
    Returns a Pytorch3D renderer for point clouds.

    Args:
        image_size (int): The rendered image size.
        device (torch.device): The torch device to use (CPU or GPU). If not specified,
            will automatically use GPU if available, otherwise CPU.
        radius (float): The radius of the rendered point in NDC.
        background_color (tuple): The background color of the rendered image.

    Returns:
        PointsRenderer.
    """
    if device is None:
        if torch.cuda.is_available():
            device = torch.device("cuda:0")
        else:
            device = torch.device("cpu")
    raster_settings = PointsRasterizationSettings(image_size=image_size, radius=radius,)
    renderer = PointsRenderer(
        rasterizer=PointsRasterizer(raster_settings=raster_settings),
        compositor=AlphaCompositor(background_color=background_color),
    )
    return renderer

In [15]:
from PIL import Image, ImageDraw
import imageio


def render_360_pc(point_cloud, image_size=256, output_path='images/q_5-1_pc1.gif',
 num_views=12, fps=15, elev=10, dist=7, device=None, background_color=(1, 1, 1)):
 #, rotate_R=False
    if device is None:
        device = torch.device("cuda:0")

    renderer = get_points_renderer(image_size=image_size, background_color=background_color)

    angles = np.linspace(-180, 180, num_views, endpoint=False)
    images = []
    for i in range(num_views):
        R, T = pytorch3d.renderer.look_at_view_transform(
        dist=dist,
        elev=elev,
        azim=angles[i],
    )
        # rotate upside down
        # if rotate_R:
        #     R = pytorch3d.transforms.euler_angles_to_matrix(torch.Tensor([0, 0, np.pi]), "XYZ") @ R

        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=R,
        T=T,
        device=device
    )

        rend = renderer(point_cloud, cameras=cameras)
        rend = rend[0, ..., :3].cpu().numpy()

        image = Image.fromarray((rend * 255).astype(np.uint8))
        draw = ImageDraw.Draw(image)
        draw.text((20, 20), f"angle: {angles[i]:.0f}", fill=(255, 0, 0))
        images.append(np.array(image))
    imageio.mimsave(output_path, images, fps=fps)

In [16]:
!ls

1_b.gif			      q_5-1_pc_union.gif
2.1.gif			      q_5-2.gif
3_b.gif			      rendering_generic_3d_representations.ipynb
data			      rendering_parametric_and_implicit.ipynb
deform_form.ipynb	      rendering_point_clouds_from_rgbd.ipynb
plotly_rendered_meshes.ipynb  rendering_texture_obj.ipynb
q_5-1_pc1.gif		      render_tetrahedron_and_retexturing.ipynb
q_5-1_pc2.gif		      render_texture_mesh_and_dolly_zoom.ipynb


In [17]:
x = (2 + 1 * torch.cos(Theta)) * torch.cos(Phi)
y = (2 + 1 * torch.cos(Theta)) * torch.sin(Phi)
z = 1 * torch.sin(Theta)

points = torch.stack((x.flatten(), y.flatten(), z.flatten()), dim=1)
color = (points - points.min()) / (points.max() - points.min())

torus_point_cloud = pytorch3d.structures.Pointclouds(
points=[points], features=[color],
).to(device)

image_size=512


render_360_pc(torus_point_cloud, image_size=image_size, output_path='q_5-2.gif', fps=7, dist=7, device=device)

In [18]:
!pip install pymcubes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.3/274.3 kB 3.6 MB/s eta 0:00:00


In [29]:
from pytorch3d.renderer import (
    AlphaCompositor,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    PointsRasterizationSettings,
    PointsRenderer,
    PointsRasterizer,
    HardPhongShader,
)


def get_mesh_renderer(image_size=512, lights=None, device=None):
    """
    Returns a Pytorch3D Mesh Renderer.

    Args:
        image_size (int): The rendered image size.
        lights: A default Pytorch3D lights object.
        device (torch.device): The torch device to use (CPU or GPU). If not specified,
            will automatically use GPU if available, otherwise CPU.
    """
    if device is None:
        if torch.cuda.is_available():
            device = torch.device("cuda:0")
        else:
            device = torch.device("cpu")
    raster_settings = RasterizationSettings(
        image_size=image_size, blur_radius=0.0, faces_per_pixel=1,
    )
    renderer = MeshRenderer(
        rasterizer=MeshRasterizer(raster_settings=raster_settings),
        shader=HardPhongShader(device=device, lights=lights),
    )
    return renderer

In [20]:
def render_360_mesh(mesh, image_size=256, output_path='images/q_5-3.gif', num_views=12, fps=7, elev=10, dist=7, device=None):
    if device is None:
        device = device
    renderer = get_mesh_renderer(image_size=image_size, device=device)
    lights = pytorch3d.renderer.PointLights(location=[[0, 0.0, -4.0]], device=device,)

    angles = np.linspace(-180, 180, num_views, endpoint=False)
    images = []
    for i in range(num_views):
        R, T = pytorch3d.renderer.look_at_view_transform(
        dist=dist,
        elev=elev,
        azim=angles[i],
    )

        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=R,
        T=T,
        device=device
    )

        rend = renderer(mesh, cameras=cameras, lights=lights)
        rend = rend[0, ..., :3].cpu().numpy()

        image = Image.fromarray((rend * 255).astype(np.uint8))
        draw = ImageDraw.Draw(image)
        draw.text((20, 20), f"angle: {angles[i]:.0f}", fill=(255, 0, 0))
        images.append(np.array(image))
    imageio.mimsave(output_path, images, fps=fps)

In [36]:
import mcubes
import numpy as np


voxel_size=64
#now for the part of implicit representation

min_value = -4.1
max_value = 4.1
X, Y, Z = torch.meshgrid([torch.linspace(min_value, max_value, voxel_size)] * 3)
# voxels = X ** 2 + Y ** 2 + Z ** 2 - 1
voxels=(torch.sqrt(X ** 2 + Y ** 2) - 2)**2 + Z ** 2 - 1**2


# print(voxels.shape)
#voxel is [depth, height, width]

# voxels=voxels[:,:,:32]

#note here the marching_cube algorithm can convert from implicit(parameter fucntion) to explicit(mesh)
vertices, faces= mcubes.marching_cubes(mcubes.smooth(voxels), isovalue=0)


vertices=torch.tensor(vertices.astype(np.float32)).to(device)
faces=torch.tensor(faces.astype(np.int32)).to(device)


vertices = (vertices / voxel_size) * (max_value - min_value) + min_value
textures = (vertices - vertices.min()) / (vertices.max() - vertices.min())
textures = pytorch3d.renderer.TexturesVertex(vertices.unsqueeze(0))
mesh = pytorch3d.structures.Meshes([vertices], [faces], textures=textures).to(
    device
)


fig = plot_scene({
    "subplot1": {
        "cow_mesh": mesh
    }
})
fig.show()

render_360_mesh(mesh, output_path='q_5-3.gif',device=device)


